Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 9.0.2

torch: 2.6.0+cu126



# LeNet-5 CIFAR10 Classifier

This notebook implements the classic LeNet-5 convolutional network [1] and applies it to MNIST digit classification.  
本笔记本实现了经典的LeNet-5卷积神经网络[1]并将其应用于MNIST数字分类。

The basic architecture is shown in the figure below:  
基本架构如下图所示：

![](../images/lenet/lenet-5_1.jpg)  

LeNet-5 is commonly regarded as the pioneer of convolutional neural networks, consisting of a very simple architecture (by modern standards).  
LeNet-5通常被认为是卷积神经网络的开创者，它由一个非常简单的架构组成（按现代标准来看）。

In total, LeNet-5 consists of only 7 layers. 3 out of these 7 layers are convolutional layers (C1, C3, C5), which are connected by two average pooling layers (S2 & S4).  
总的来说，LeNet-5只有7层，其中3层是卷积层（C1、C3、C5），它们通过两个平均池化层（S2和S4）连接。

The penultimate layer is a fully connected layer (F6), which is followed by the final output layer.  
倒数第二层是全连接层（F6），之后是最终的输出层。

The additional details are summarized below:  
额外的细节总结如下：

- All convolutional layers use 5x5 kernels with stride 1.  
  所有卷积层都使用5x5的卷积核，步幅为1。

- The two average pooling (subsampling) layers are 2x2 pixels wide with stride 1.  
  两个平均池化（子采样）层宽度为2x2像素，步幅为1。

- Throughout the network, tanh sigmoid activation functions are used. (**In this notebook, we replace these with ReLU activations**)  
  网络中使用的是tanh sigmoid激活函数。（**在本笔记本中，我们将其替换为ReLU激活函数**）

- The output layer uses 10 custom Euclidean Radial Basis Function neurons for the output layer. (**In this notebook, we replace these with softmax activations**)  
  输出层使用10个自定义的欧几里得径向基函数神经元作为输出层。（**在本笔记本中，我们将其替换为softmax激活函数**）

- The input size is 32x32; here, we rescale the MNIST images from 28x28 to 32x32 to match this input dimension. Alternatively, we would have to change the  
  输入大小是32x32；在这里，我们将MNIST图像从28x28重新缩放到32x32以匹配此输入维度。否则，我们将不得不更改

achieve error rate below 1% on the MNIST data set, which was very close to the state of the art at the time (produced by a boosted ensemble of three LeNet-4 networks).  
在MNIST数据集上实现低于1%的错误率，这在当时接近最先进的技术水平（由三个LeNet-4网络的增强集成生成）。

### References  
### 参考文献

- [1] Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Gradient-based learning applied to document recognition. Proceedings of the IEEE, november 1998.  
- [1] Y. LeCun, L. Bottou, Y. Bengio和P. Haffner. 基于梯度的学习应用于文档识别. IEEE会议录，1998年11月。

## Imports

In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [3]:
##########################
### 配置
##########################

# 超参数
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# 网络结构
NUM_FEATURES = 32*32
NUM_CLASSES = 10

# 其他
DEVICE = "cuda:0"
GRAYSCALE = False

### MNIST Dataset

In [4]:
##########################
### CIFAR-10 数据集
##########################


# 注意 transforms.ToTensor() 会将输入图像缩放到 0-1 范围
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=8,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

# 检查数据集
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

# 再次检查数据集
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


In [5]:
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


In [6]:
##########################
### 模型
##########################


class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6*in_channels, 16*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5*in_channels, 120*in_channels),
            nn.Tanh(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.Tanh(),
            nn.Linear(84*in_channels, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [7]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, GRAYSCALE)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

## Training

In [8]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### 前向传播和反向传播
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### 更新模型参数
        optimizer.step()
        
        ### 日志记录
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0391 | Cost: 2.3115
Epoch: 001/010 | Batch 0050/0391 | Cost: 1.7790
Epoch: 001/010 | Batch 0100/0391 | Cost: 1.5894
Epoch: 001/010 | Batch 0150/0391 | Cost: 1.5928
Epoch: 001/010 | Batch 0200/0391 | Cost: 1.3675
Epoch: 001/010 | Batch 0250/0391 | Cost: 1.4105
Epoch: 001/010 | Batch 0300/0391 | Cost: 1.2797
Epoch: 001/010 | Batch 0350/0391 | Cost: 1.6213
Epoch: 001/010 | Train: 53.020%
Time elapsed: 0.04 min
Epoch: 002/010 | Batch 0000/0391 | Cost: 1.4377
Epoch: 002/010 | Batch 0050/0391 | Cost: 1.1872
Epoch: 002/010 | Batch 0100/0391 | Cost: 1.3558
Epoch: 002/010 | Batch 0150/0391 | Cost: 1.1609
Epoch: 002/010 | Batch 0200/0391 | Cost: 1.2661
Epoch: 002/010 | Batch 0250/0391 | Cost: 1.0852
Epoch: 002/010 | Batch 0300/0391 | Cost: 1.1308
Epoch: 002/010 | Batch 0350/0391 | Cost: 1.2074
Epoch: 002/010 | Train: 61.494%
Time elapsed: 0.07 min
Epoch: 003/010 | Batch 0000/0391 | Cost: 1.1985
Epoch: 003/010 | Batch 0050/0391 | Cost: 1.1419
Epoch: 003/010 | Batch 010

## Evaluation

In [9]:
with torch.set_grad_enabled(False): # 在推理过程中禁用梯度计算，节省内存
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 66.81%


In [10]:
%watermark -iv

PIL        : 11.1.0
torchvision: 0.21.0+cu126
matplotlib : 3.10.1
numpy      : 1.26.4
torch      : 2.6.0+cu126
pandas     : 2.2.3

